# Generic dfs processing
Tools and methods that applies to any type of dfs files. 

The generic tools are useful for common data processing tasks, where detailed configuration is not necessary. 

* mikeio.read()
* mikeio.generic: methods that read any dfs file and outputs a new dfs file of the same type
   - concat: Concatenates files along the time axis  
   - scale: Apply scaling to any dfs file
   - sum: Sum two dfs files 
   - diff: Calculate difference between two dfs files
   - extract: Extract timesteps and/or items to a new dfs file

The generic methods works on larger-than-memory files as they process one time step at a time. This can however make them in-efficient for dfs0 processing! 

## Concatenation

In [ ]:
import matplotlib.pyplot as plt
import mikeio

Take a look at these two files with overlapping timesteps.

In [ ]:
t1 = mikeio.read("data/tide1.dfs1")
t1

In [ ]:
t2 = mikeio.read("data/tide2.dfs1")
t2

Plot one of the points along the line.

In [ ]:
plt.plot(t1.time,t1.data[0][:,1], label="File 1")
plt.plot(t2.time,t2.data[0][:,1],'k+', label="File 2")
plt.legend()

In [ ]:
import mikeio.generic

mikeio.generic.concat(infilenames=["data/tide1.dfs1",
                                   "data/tide2.dfs1"],
                     outfilename="concat.dfs1")

In [ ]:
c = mikeio.read("concat.dfs1")
plt.plot(c.time,c.data[0][:,1])
c

## Extract time steps or items

In [ ]:
infile = "data/tide1.dfs1"
mikeio.generic.extract(infile, "extracted.dfs1", start='2019-01-02')

In [ ]:
e = mikeio.read("extracted.dfs1")
e

In [ ]:
infile = "data/oresund_vertical_slice.dfsu"
mikeio.generic.extract(infile, "extracted.dfsu", items='Salinity', end=-2)

In [ ]:
e = mikeio.read("extracted.dfsu")
e

## Scaling

Adding a constant e.g to adjust datum

In [ ]:
ds = mikeio.read("data/gebco_sound.dfs2")
plt.imshow(ds['Elevation'][0])
plt.colorbar();

In [ ]:
ds['Elevation'][0,104,131]

This is the processing step.

In [ ]:
mikeio.generic.scale("data/gebco_sound.dfs2","gebco_sound_local_datum.dfs2",offset=-2.1)

In [ ]:
ds2 = mikeio.read("gebco_sound_local_datum.dfs2")
plt.imshow(ds2['Elevation'][0])
plt.colorbar();

In [ ]:
ds2['Elevation'][0,104,131]

### Spatially varying correction

In [ ]:
import numpy as np
factor = np.ones_like(ds['Elevation'][0])
factor.shape

Add some spatially varying factors, exaggerated values for educational purpose.

In [ ]:
factor[:,0:100] = 5.3
factor[0:40,] = 0.1
factor[150:,150:] = 10.7
plt.imshow(factor)
plt.colorbar();

The 2d array must first be flipped upside down and then converted to a 1d vector using [numpy.ndarray.flatten](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) to match how data is stored in dfs files.

In [ ]:
factor_ud = np.flipud(factor)
factor_vec  = factor_ud.flatten()
mikeio.generic.scale("data/gebco_sound.dfs2","gebco_sound_spatial.dfs2",factor=factor_vec)

In [ ]:
ds3 = mikeio.read("gebco_sound_spatial.dfs2")
plt.imshow(ds3['Elevation'][0])
plt.colorbar()
plt.title("Spatial correction applied to dfs2");

## Clean up

In [ ]:
import os
os.remove("concat.dfs1")
os.remove("extracted.dfs1")
os.remove("extracted.dfsu")
os.remove("gebco_sound_local_datum.dfs2")
os.remove("gebco_sound_spatial.dfs2")